In [1]:
import numpy as np
import fasttext
import pandas as pd
import random
import itertools
from resources.tokTT import CommentTokenizer as CT
from resources.basicIO import InputOutput as IO
from resources.basicIO import InputOutput as IO
from resources.filterLang import FilterLanguage as FL
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
import fasttext
import pandas as pd
from scipy import spatial
from scipy.spatial import distance
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LassoLars
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import copy
import scipy


[nltk_data] Downloading package wordnet to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess Corpus and Random Sample

In [2]:
raw_corpus = IO.load_text('datasets/corpus.txt')
tokenized_corpus = CT.cleaned('datasets/corpus.txt')
IO.save_text('datasets/tokenized_corpus.txt',tokenized_corpus)

### Make Fasttext Unsupervised Model

In [2]:
model_2 = fasttext.train_unsupervised(
    input="datasets/tokenized_corpus.txt", lr=0.01, epoch=50, wordNgrams=3, dim=300)
model_2.save_model("models/ft_unsupervised_N_3.bin")


### Make dataframes

In [4]:
# corpus
df_dict = {'raw_comment': raw_corpus, 'tokenized_comment': tokenized_corpus}
df_corpus = pd.DataFrame(df_dict)
df_corpus.to_csv('datasets/corpus_data.csv', index=False)

# random sample
text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
text_labels = list(map(str, map(int, text_labels)))
text_TK = [CT.tokenize(x) for x in text]

df_dict = {'raw_comment': text,
           'tokenized_comment': text_TK, 'label': text_labels}
df_sample = pd.DataFrame(df_dict)
df_sample.to_csv('datasets/random_sample_data.csv', index=False)


### Remove Unnecessary Comments

In [5]:
# remove unnecessary comments
df_sample = df_sample.drop([x for x in range(len(df_sample)) if int(
    df_sample['label'][x]) != 0 and int(df_sample['label'][x]) != 1])

In [6]:
df_sample.tail()


,raw_comment,tokenized_comment,label
4477,all the flags are sikh religious flags. they d...,all the flag be sikh religious flag they do no...,1
4478,Salute the farmers,salute the farmer,0
4479,Let's also remember how many farmers lost thei...,let be also remember how many farmer lose thei...,0
4480,ABP news you guys dont have heart 8 farmers go...,abp news you guy dont have heart 8 farmer get ...,0
4481,These new laws actually give the power to the ...,these new law actually give the power to the f...,1


In [7]:
df_sample['label'].value_counts()


0    1846
1    1408
Name: label, dtype: int64

### Train test split

In [8]:

X_train, X_test, y_train, y_test = train_test_split(df_sample['tokenized_comment'],
                                                    df_sample['label'], test_size=0.2,
                                                    random_state=42,
                                                    stratify=df_sample['label'])


In [9]:
print('X_train: ' ,len(X_train))
print('X_test: ' ,len(X_test))

X_train:  2603
X_test:  651


### Classification

In [10]:
class FastTextTransformer(BaseEstimator, TransformerMixin):
    """ Convert texts into their mean fastText vectors """

    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.stack([np.mean([self.model[w] for w in text.split()], 0) for text in X])


def classify(small_model, predictor, lines, Y):
    classifier = make_pipeline(
        FastTextTransformer(model=small_model),
        predictor
    ).fit(
        lines,
        Y
    )
    return classifier


### Classifier

In [11]:
# Logistic Regression
LR_Normal = classify(model_2, LogisticRegression(
    random_state=1), X_train, y_train)
# SVM
SVM_Normal = classify(model_2, svm.SVC(), X_train, y_train)

models = []
models.append(('LR Normal N=2', LR_Normal))
models.append(('SVM Normal N=2', SVM_Normal))


In [12]:
file = open('results/output_better.txt', 'w+')
file.close()

outfile = open("results/output_better.txt", "a")
for i, v in models:
    print(i)
    accuracy = metrics.accuracy_score(y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(y_test, v.predict(X_test))
    print('========= {} Model Test Results ==========='.format(i), file=outfile)
    print(' ', file=outfile)
    print("Model Accuracy:" "\n", accuracy, file=outfile)
    print(accuracy)
    print(' ', file=outfile)
    print("Confusion matrix:" "\n", confusion_matrix, file=outfile)
    print(' ', file=outfile)
outfile.close()


LR Normal N=2
0.7603686635944701
SVM Normal N=2
0.7619047619047619


### Active Learning

In [13]:
X_seed, X_expand, y_seed, y_expand = train_test_split(X_train,
                                                      y_train, test_size=0.99,
                                                      random_state=41,
                                                      stratify=y_train)


In [14]:
print('X_seed: ',len(X_seed))
print('X_expand: ',len(X_expand))

X_seed:  26
X_expand:  2577


In [15]:
y_seed.value_counts()


0    15
1    11
Name: label, dtype: int64

In [16]:
df_dict = {'seed_tokenized': X_seed,
           'label': y_seed}
df_seed = pd.DataFrame(df_dict)
df_seed.to_csv('datasets/seed_data.csv', index=False)

df_dict = {'expansion_tokenized': X_expand,
           'label': y_expand}
df_expand = pd.DataFrame(df_dict)
df_seed.to_csv('datasets/expand_data.csv', index=False)


### Cosine Similarity And Nearest Neighbors

In [17]:
def score(model, line, k):
    """ Returns a vector containing nearest neighbor scores w.r.t. all
        words in the model """
    # words contains all the words in the corpus
    lst1 = model.get_nearest_neighbors(line, k)
    v1 = []
    l1 = [x[1] for x in lst1]
    l10 = [x[0] for x in lst1]
    for i in range(len(model.words)):
        try:
            v1.append(l10[l1.index(model.words[i])])
        except:
            v1.append(0)
    return v1


def NN(model: fasttext.FastText._FastText, line: str, K):
    """ Returns k fasttext nearest neighbors of a given string """
    return model.get_nearest_neighbors(line, k=K)


def get_NN(model: fasttext.FastText._FastText, lines: list, k: int):
    """ Returns k nearest neighbor scores of multiple strings"""
    scores = []
    for line in lines:
        scores.append(score(model, line, k))
    return scores

def cos_sim(a: np.array, b: np.array):
    """ Returns cosine similarity of two 1d arrays """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if(norm_a * norm_b == 0.0):
        return dot_product / (norm_a * norm_b + 0.001)
    return dot_product / (norm_a * norm_b)


def sim(x: np.array, y: np.array, sim_type: str):
    if(sim_type == 'cosine_sim'):
        return cos_sim(x, y)


def sim_matrix(A: np.array, B: np.array, sim_type: str):
    """ find similarity score matrix between A and B. 
        A,B: 2d matrix of embeddings/nearest neighbor scores.
        sim_type: String denoting type of similarity.
    """
    m, p = A.shape
    p, n = B.shape
    C = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            C[i][j] = sim(A[i, :], B[:, j], sim_type)
    return C


### Expansion Code (Random Sampling)

In [18]:
def Expand_R(model: fasttext.FastText._FastText, 
             seed_set_tokenised: list, 
             seed_set_label: list, 
             expansion_tokenised: list, 
             expansion_set_labels: list, 
             batch_size: int, 
             k_neighbors: int, 
             random_rate: float):

    """
    Takes seed set and and expands the set using expansion_tokenised.
    Batch size: no. of texts to be inserted in one go
    k_neighbors: no. of neighbors for getting cosine similarity
    random_rate: fraction of amount taken for random sampling
    """
             
    seed_TK = copy.deepcopy(seed_set_tokenised)
    seed_labels = copy.deepcopy(seed_set_label)
    count = len(expansion_set_labels)
    M = np.arange(0, count, batch_size)
    cnt = int(random_rate * batch_size)
    
    expansion_predicted_labels = []
    expansion_true_labels = []
    expansion_accuracy = []

    for i in range(1, len(M)):

        print(M[i], end=' ')

        # select batchwise expansion text
        exp_TK = expansion_tokenised[M[i-1]:M[i]]
        exp_labels = expansion_set_labels[M[i-1]:M[i]]

        # nearest neighbors
        seed_NN = get_NN(model, seed_TK, k_neighbors)
        exp_NN = get_NN(model, exp_TK, k_neighbors)

        A = np.array(seed_NN)
        B = np.array(exp_NN).T
        C = sim_matrix(A, B, "cosine_sim")

        # find rowwise (seed) index of highest similarity
        Y_ind = np.argmax(C, axis=0)
        # get labels
        Y = [seed_labels[x] for x in Y_ind]

        if(random_rate == 0.0):
            # no random sampling
            pass
        else:
            # random sampling
            Y_r = random.sample(range(0,len(Y)), cnt)
            for j in Y_r:
                Y[j] = exp_labels[j]

        # calc. expansion accuracy
        expansion_predicted_labels.extend(Y)
        expansion_true_labels.extend(exp_labels)
        expansion_accuracy.append(metrics.accuracy_score(exp_labels, Y))

        # expand seed set
        seed_labels.extend(Y)
        seed_TK.extend(exp_TK)

    return seed_TK, seed_labels, expansion_true_labels, expansion_predicted_labels, expansion_accuracy


In [19]:
X_train_exp, y_train_exp, et,ep, expansion_accuracy = Expand_R(model_2,
                                            list(X_seed),
                                            list(y_seed),
                                            list(X_expand),
                                            list(y_expand),
                                            400,
                                            40,
                                            0.2
                                            )


400 800 1200 1600 2000 2400 

In [20]:
print(metrics.accuracy_score(et, ep))
print(expansion_accuracy)

0.6120833333333333
[0.5825, 0.585, 0.64, 0.62, 0.615, 0.63]


In [21]:
# Logistic Regression
LR_Normal_r = classify(model_2, LogisticRegression(
    random_state=1), X_train_exp, y_train_exp)
# SVM
SVM_Normal_r = classify(model_2, svm.SVC(), X_train_exp, y_train_exp)

models_r = []
models_r.append(('LR Normal N=2', LR_Normal_r))
models_r.append(('SVM Normal N=2', SVM_Normal_r))


In [22]:
file = open('results/output_random.txt', 'w+')
file.close()

outfile = open("results/output_random.txt", "a")
for i, v in models_r:
    print(i)
    accuracy = metrics.accuracy_score(y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(y_test, v.predict(X_test))
    print('========= {} Model Test Results ==========='.format(i), file=outfile)
    print(' ', file=outfile)
    print("Model Accuracy:" "\n", accuracy, file=outfile)
    print(accuracy)
    print(' ', file=outfile)
    print("Confusion matrix:" "\n", confusion_matrix, file=outfile)
    print(' ', file=outfile)
outfile.close()


LR Normal N=2
0.6082949308755761
SVM Normal N=2
0.5775729646697388


### Uncertainty Sampling

In [23]:
def Expand_U(model: fasttext.FastText._FastText, 
             algorithm: object, 
             seed_set_tokenised: list, 
             seed_set_label: list, 
             expansion_tokenised: list,
             expansion_set_labels: list, 
             batch_size: int,
             countMax: int):
    """ Uncertainty sampling.
    Expand seed set using expansion_set based on lowest confidance scores.
    max_threshold: max. probability for uncertainty selection """

    seed_TK = copy.deepcopy(seed_set_tokenised)
    seed_labels = copy.deepcopy(seed_set_label)
    count = len(expansion_set_labels)
    M = np.arange(0, count, batch_size)

    # exp_TK_certain will be the list of comments having high proba score
    exp_TK_certain = []
    exp_TK_certain_labels = []


    for i in range(1, len(M)):

        print(M[i], end=' ')

        exp_TK = expansion_tokenised[M[i-1]:M[i]]
        exp_labels = expansion_set_labels[M[i-1]:M[i]]

        # take A as training and B as test and store probs in C
        small_model = classify(model, algorithm, seed_TK, seed_labels)
        # store classwise prob. scores
        C = small_model.predict_proba(exp_TK)
        # Uncertainty sampling scores
        C_abs_diff = [(abs(x[0] - x[1])) for x in C]

        # Sort lists in ascending order of probabilities from C_abs_diff
        sorted_lists = sorted(zip(exp_labels, exp_TK, C, C_abs_diff), key=lambda x: x[3])
        exp_labels, exp_TK, C_sorted, score = [[x[i] for x in sorted_lists] for i in range(4)]

        Y_uncertain = []
        exp_TK_uncertain = []
        for j in range(len(C_sorted)):
            max_value = max(C_sorted[j])
            max_index = str(np.argmax(C_sorted[j]))

            # label the comments whose score is less than threshold
            if(j <= countMax):

                exp_TK_uncertain.append(exp_TK[j])
                Y_uncertain.append(exp_labels[j])
            else:
                exp_TK_certain.append(exp_TK[j])
                exp_TK_certain_labels.append(exp_labels[j])

        # expand the seed set
        seed_labels.extend(Y_uncertain)
        seed_TK.extend(exp_TK_uncertain)


    return seed_TK, seed_labels, exp_TK_certain, exp_TK_certain_labels


In [24]:
X_uncertain_exp_u, y_uncertain_exp_u, X_certain_exp_u, y_certain_exp_u = Expand_U(model_2,
                                                                          LogisticRegression(
                                                                              random_state=1),
                                                                          list(X_seed),
                                                                          list(y_seed),
                                                                          list(X_expand),
                                                                          list(y_expand),
                                                                          20,
                                                                          5
                                                                          )


20 40 60 80 100 120 140 160 180 200 220 240 260 280 300 320 340 360 380 400 420 440 460 480 500 520 540 560 580 600 620 640 660 680 700 720 740 760 780 800 820 840 860 880 900 920 940 960 980 1000 1020 1040 1060 1080 1100 1120 1140 1160 1180 1200 1220 1240 1260 1280 1300 1320 1340 1360 1380 1400 1420 1440 1460 1480 1500 1520 1540 1560 1580 1600 1620 1640 1660 1680 1700 1720 1740 1760 1780 1800 1820 1840 1860 1880 1900 1920 1940 1960 1980 2000 2020 2040 2060 2080 2100 2120 2140 2160 2180 2200 2220 2240 2260 2280 2300 2320 2340 2360 2380 2400 2420 2440 2460 2480 2500 2520 2540 2560 

In [25]:
print(len(X_uncertain_exp_u))
print(len(X_certain_exp_u))

794
1792


In [26]:
# Logistic Regression
LR_Normal_u = classify(model_2, LogisticRegression(
    random_state=1), X_uncertain_exp_u, y_uncertain_exp_u)
# SVM
SVM_Normal_u = classify(model_2, svm.SVC(), X_uncertain_exp_u, y_uncertain_exp_u)

# GNB
GNB_Normal_u = classify(model_2, GaussianNB(), X_uncertain_exp_u, y_uncertain_exp_u)

# DT
DT_Normal_u = classify(model_2, DecisionTreeClassifier(),X_uncertain_exp_u, y_uncertain_exp_u)

models_u = []
models_u.append(('LR Normal N=2', LR_Normal_u))
models_u.append(('SVM Normal N=2', SVM_Normal_u))
models_u.append(('GNB Normal N=2', GNB_Normal_u))
models_u.append(('DT Normal N=2', DT_Normal_u))


In [27]:
file = open('results/output_uncertain.txt', 'w+')
file.close()

outfile = open("results/output_uncertain.txt", "a")
for i, v in models_u:
    print(i)
    accuracy = metrics.accuracy_score(y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(y_test, v.predict(X_test))
    print('========= {} Model Test Results ==========='.format(i), file=outfile)
    print(' ', file=outfile)
    print("Model Accuracy:" "\n", accuracy, file=outfile)
    print(accuracy)
    print(' ', file=outfile)
    print("Confusion matrix:" "\n", confusion_matrix, file=outfile)
    print(' ', file=outfile)
outfile.close()


LR Normal N=2
0.728110599078341
SVM Normal N=2
0.7419354838709677
GNB Normal N=2
0.6835637480798771
DT Normal N=2
0.5791090629800307


### Certainty Sampling

In [28]:
X_train_certain, y_train_certain, expansion_true_labels, expansion_predicted_labels, expansion_accuracy = Expand_R(model_2,
                                            X_uncertain_exp_u,
                                            y_uncertain_exp_u,
                                            X_certain_exp_u,
                                            y_certain_exp_u,
                                            20,
                                            40,
                                            0.2
                                            )


20 40 60 80 100 120 140 160 180 200 220 240 260 280 300 320 340 360 380 400 420 440 460 480 500 520 540 560 580 600 620 640 660 680 700 720 740 760 780 800 820 840 860 880 900 920 940 960 980 1000 1020 1040 1060 1080 1100 1120 1140 1160 1180 1200 1220 1240 1260 1280 1300 1320 1340 1360 1380 1400 1420 1440 1460 1480 1500 1520 1540 1560 1580 1600 1620 1640 1660 1680 1700 1720 1740 1760 1780 

In [29]:
print(expansion_accuracy)

[0.7, 0.65, 0.65, 0.75, 0.75, 0.8, 0.6, 0.6, 0.7, 0.8, 0.8, 0.75, 0.75, 0.7, 0.65, 0.6, 0.75, 0.55, 0.85, 0.55, 0.85, 0.65, 0.7, 0.75, 0.6, 0.9, 0.75, 0.7, 0.65, 0.7, 0.75, 0.6, 0.75, 0.7, 0.75, 0.55, 0.75, 0.85, 0.65, 0.95, 0.7, 0.75, 0.75, 0.6, 0.85, 0.7, 0.6, 0.75, 0.65, 0.75, 0.8, 0.65, 0.8, 0.8, 0.65, 0.7, 0.75, 0.7, 0.75, 0.7, 0.8, 0.6, 0.6, 0.55, 0.7, 0.55, 0.75, 0.65, 0.75, 0.85, 0.65, 0.8, 0.7, 0.7, 0.6, 0.9, 0.7, 0.75, 0.4, 0.6, 0.75, 0.75, 0.6, 0.75, 0.75, 0.8, 0.65, 0.75, 0.7]


In [30]:
# Logistic Regression
LR_Normal_cc = classify(model_2, LogisticRegression(
    random_state=1), X_train_certain, y_train_certain)
# SVM
SVM_Normal_cc = classify(model_2, svm.SVC(), X_train_certain, y_train_certain)

models_cc = []
models_cc.append(('LR Normal N=2', LR_Normal_cc))
models_cc.append(('SVM Normal N=2', SVM_Normal_cc))


In [31]:
file = open('results/output_certain.txt', 'w+')
file.close()

outfile = open("results/output_certain.txt", "a")
for i, v in models_cc:
    print(i)
    accuracy = metrics.accuracy_score(y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(y_test, v.predict(X_test))
    print('========= {} Model Test Results ==========='.format(i), file=outfile)
    print(' ', file=outfile)
    print("Model Accuracy:" "\n", accuracy, file=outfile)
    print(accuracy)
    print(' ', file=outfile)
    print("Confusion matrix:" "\n", confusion_matrix, file=outfile)
    print(' ', file=outfile)
outfile.close()


LR Normal N=2
0.7311827956989247
SVM Normal N=2
0.7327188940092166
